# RNN, GRU, LSTM

reference : https://www.youtube.com/watch?v=Gl2WXLIMvKA&list=PLhhyoLH6IjfxeoooqP9rhU3HJIAVAJ3Vz&index=5

In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# BASE

In [6]:
input_size = 784
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 1

In [7]:
class NN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, num_classes)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [8]:
train_dataset = datasets.MNIST(root = './content',
                              train = True,
                              transform = transforms.ToTensor(),
                              download = False)

test_dataset = datasets.MNIST(root = './content',
                              train = False,
                              transform = transforms.ToTensor(),
                              download = False)

train_loader = DataLoader(dataset = train_dataset,
                         batch_size=batch_size,
                         shuffle = True)

test_loader = DataLoader(dataset = test_dataset,
                         batch_size=batch_size,
                         shuffle = True)

In [10]:
model = NN(input_size=input_size, num_classes=num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [14]:
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        #Get data
        data = data.to(device=device)
        targets = targets.to(device=device)
        
        #Get reshape data
        data = data.reshape(data.shape[0], -1)
        
        scores = model(data)
        loss = criterion(scores, targets)
        
        optimizer.zero_grad()
        loss.backward()
        
        optimizer.step()

In [16]:
def check_accuracy(loader, model):
    if loader.dataset.train:
        print('check train data')
        
    else:
        print('check test data')
        
    num_correct = 0
    num_samples = 0
    model.eval()
    
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            x = x.reshape(x.shape[0],-1)
            
            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
            
        print(f'Got {num_correct} / {num_samples} with accuracy \
        {float(num_correct)/float(num_samples)*100:.2f}')

    model.train()
    
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

check train data
Got 56032 / 60000 with accuracy         93.39
check test data
Got 9315 / 10000 with accuracy         93.15


# BASE + Batchnorm

In [17]:
input_size = 784
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 1

class NN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, num_classes)
        self.batch = nn.BatchNorm1d(50)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.batch(x)
        x = self.fc2(x)
        return x

    
model = NN(input_size=input_size, num_classes=num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        #Get data
        data = data.to(device=device)
        targets = targets.to(device=device)
        
        #Get reshape data
        data = data.reshape(data.shape[0], -1)
        
        scores = model(data)
        loss = criterion(scores, targets)
        
        optimizer.zero_grad()
        loss.backward()
        
        optimizer.step()
        
        
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

check train data
Got 57550 / 60000 with accuracy         95.92
check test data
Got 9547 / 10000 with accuracy         95.47


# RNN

In [23]:
# Make RNN

input_size = 28
sequence_length = 28
num_layers = 2
hidden_size = 256
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 2

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, 
                          batch_first=True)
        self.fc = nn.Linear(hidden_size*sequence_length, num_classes)
        
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        #Forward
        out, _ = self.rnn(x, h0)
        out = out.reshape(out.shape[0], -1)
        out = self.fc(out)
        return out
    
model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        #Get data
        data = data.to(device=device).squeeze(1)
        targets = targets.to(device=device)
        
        scores = model(data)
        loss = criterion(scores, targets)
        
        optimizer.zero_grad()
        loss.backward()
        
        optimizer.step()
        
def check_accuracy(loader, model):
    if loader.dataset.train:
        print('check train data')
        
    else:
        print('check test data')
        
    num_correct = 0
    num_samples = 0
    model.eval()
    
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device).squeeze(1)
            y = y.to(device=device)
            
            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
            
        print(f'Got {num_correct} / {num_samples} with accuracy \
        {float(num_correct)/float(num_samples)*100:.2f}')

    model.train()
        
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

check train data
Got 58074 / 60000 with accuracy         96.79
check test data
Got 9650 / 10000 with accuracy         96.50


# GRU

In [27]:
# Make RNN

input_size = 28
sequence_length = 28
num_layers = 2
hidden_size = 256
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 2

class GRU(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(GRU, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(input_size, hidden_size, num_layers, 
                          batch_first=True)
        self.fc = nn.Linear(hidden_size*sequence_length, num_classes)
        
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        #Forward
        out, _ = self.gru(x, h0)
        out = out.reshape(out.shape[0], -1)
        out = self.fc(out)
        return out
    
model = GRU(input_size, hidden_size, num_layers, num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        #Get data
        data = data.to(device=device).squeeze(1)
        targets = targets.to(device=device)
        
        scores = model(data)
        loss = criterion(scores, targets)
        
        optimizer.zero_grad()
        loss.backward()
        
        optimizer.step()
        
def check_accuracy(loader, model):
    if loader.dataset.train:
        print('check train data')
        
    else:
        print('check test data')
        
    num_correct = 0
    num_samples = 0
    model.eval()
    
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device).squeeze(1)
            y = y.to(device=device)
            
            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
            
        print(f'Got {num_correct} / {num_samples} with accuracy \
        {float(num_correct)/float(num_samples)*100:.2f}')

    model.train()
        
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

check train data
Got 59208 / 60000 with accuracy         98.68
check test data
Got 9823 / 10000 with accuracy         98.23


### LSTM

In [29]:
# Make RNN

input_size = 28
sequence_length = 28
num_layers = 2
hidden_size = 256
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 2

class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, 
                          batch_first=True)
        self.fc = nn.Linear(hidden_size*sequence_length, num_classes)
        
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        #Forward
        out, _ = self.lstm(x, (h0, c0))
        out = out.reshape(out.shape[0], -1)
        out = self.fc(out)
        return out
    
model = LSTM(input_size, hidden_size, num_layers, num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        #Get data
        data = data.to(device=device).squeeze(1)
        targets = targets.to(device=device)
        
        scores = model(data)
        loss = criterion(scores, targets)
        
        optimizer.zero_grad()
        loss.backward()
        
        optimizer.step()
        
def check_accuracy(loader, model):
    if loader.dataset.train:
        print('check train data')
        
    else:
        print('check test data')
        
    num_correct = 0
    num_samples = 0
    model.eval()
    
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device).squeeze(1)
            y = y.to(device=device)
            
            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
            
        print(f'Got {num_correct} / {num_samples} with accuracy \
        {float(num_correct)/float(num_samples)*100:.2f}')

    model.train()
        
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

check train data
Got 59258 / 60000 with accuracy         98.76
check test data
Got 9861 / 10000 with accuracy         98.61
